In [1]:
import json
import pandas as pd
import numpy as np

from openai import OpenAI

In [12]:
df = pd.read_csv('../data/historical_data.csv')

In [13]:
df_cleaned = df[df["topic_description"].notna() & (df["topic_description"].str.strip() != "")]

In [14]:
unique_cases = df_cleaned[["topic_title", "topic_description"]].drop_duplicates()

unique_cases = unique_cases.reset_index(drop=True)

unique_cases.rename(columns={"topic_title": "case_title", "topic_description": "case_description"}, inplace=True)

Очищу вручную данные в экселе

In [45]:
unique_cases.to_excel('cases.xlsx', index=False)

In [2]:
unique_cases = pd.read_excel('cases.xlsx')

In [5]:
client = OpenAI(
    api_key="sk-lpIBtFRXrvF8sDPBQ3lB8rTQtszRRVwj",
    base_url="https://api.proxyapi.ru/openai/v1",
)

In [6]:
def generate_required_roles(case_title, case_description):
    system_prompt = """
    У меня есть датасет кейсов с полями case_title, case_description. Мне нужно к каждому кейсу добавить поле required_roles с необходимыми ролями для выполнения кейса.
    Ниже представлен список ролей, которые могут быть необходимы в кейсах:
    - Java Backend
    - C# Backend
    - Go Backend
    - Python Backend
    - C++ Backend
    - Frontend
    - ML engineer
    - DevOps
    - Тестировщик
    - Аналитик
    - Дизайнер
    - Инженер БПЛА
    - CV engineer
    - ML Ops Engineer
    
    Тебе необходимо написать только самые необходимые роли (обычно от 4 до 5 ролей, но может быть или больше или меньше).
    """
    user_prompt = f"""
    case_title: '{case_title}'
    case_description: '{case_description}'
    
    В ответе напиши только роли, перечисляя их через запятую.
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Update with the correct model name if needed
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            max_tokens=100,
            temperature=0.5
        )
        roles = response.choices[0].message.content.strip()
        return roles
        # return roles.split("\n") if roles else []
    except Exception as e:
        print(f"Error generating roles for case '{case_title}': {e}")
        return []
    

In [7]:
required_roles_list = []

# Iterate through the rows of the DataFrame
for index, row in unique_cases.iterrows():
    roles = generate_required_roles(row["case_title"], row["case_description"])
    required_roles_list.append(roles)

In [8]:
unique_cases["required_roles"] = required_roles_list

In [9]:
unique_cases

,case_title,case_description,required_roles
0,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...","ML engineer, Python Backend, DevOps, CV engine..."
1,Оптимальное распределение задач на основе псих...,"Результат проекта: Чат-бот, способный распреде...","ML engineer, Python Backend, Аналитик, Дизайне..."
2,БАМ,Определение контура железной дороги БАМ на сп...,"ML engineer, CV engineer, Python Backend, Тест..."
3,Система автоматического анализа электронного д...,Требования: для анализа входящих неформазиован...,"ML engineer, Python Backend, Тестировщик, Анал..."
4,Анализ новостей для предсказания движения акци...,#### Цель Проекта\n\nРазработать систему анали...,"ML engineer, Python Backend, Аналитик, Дизайне..."
...,...,...,...
84,Применение сверточных нейронных сетей для сегм...,Этот проект фокусируется на использовании свер...,"ML engineer, Python Backend, Аналитик, Дизайне..."
85,Ледовая разведка в Карском море,Мониторинг ледников и ледовая разведка в Карск...,"ML engineer, Python Backend, DevOps, Аналитик,..."
86,Разработка концепта экспертной системы по пред...,Целевая аудитория: Авиационные технические отр...,"Инженер БПЛА, ML engineer, Тестировщик, Аналит..."
87,Формирование БД для Postgresql на основе ER-мо...,Нужно будет провести исследование существующих...,"C++ Backend, Python Backend, Дизайнер, Аналити..."


In [ ]:
unique_cases.to_csv('cleaned_cases_with_roles.csv', index=False)

In [16]:
unique_cases.head()

,case_title,case_description,required_roles
0,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...","ML engineer, Python Backend, DevOps, CV engine..."
1,Оптимальное распределение задач на основе псих...,"Результат проекта: Чат-бот, способный распреде...","ML engineer, Python Backend, Аналитик, Дизайне..."
2,БАМ,Определение контура железной дороги БАМ на сп...,"ML engineer, CV engineer, Python Backend, Тест..."
3,Система автоматического анализа электронного д...,Требования: для анализа входящих неформазиован...,"ML engineer, Python Backend, Тестировщик, Анал..."
4,Анализ новостей для предсказания движения акци...,#### Цель Проекта\n\nРазработать систему анали...,"ML engineer, Python Backend, Аналитик, Дизайне..."


In [17]:
df_cleaned.rename(columns={"topic_title": "case_title", "topic_description": "case_description"}, inplace=True)

/var/folders/3_/0zlq__wj47xg9llgg6qg6jzc0000gn/T/ipykernel_10613/453769300.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.rename(columns={"topic_title": "case_title", "topic_description": "case_description"}, inplace=True)


In [18]:
df_cleaned.head()

,team_id,team_title,case_title,case_description,user_fio,skills
0,81,10pandas,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Додин Даниил Дмитриевич,NaN
1,81,10pandas,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Хижний Никита Александрович,"Data Science, Computer Vision, Confluence, Dat..."
2,81,10pandas,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Папуашвили Георгий Давидович,"С++, AirFlow, Computer Vision, Confluence, Dat..."
3,81,10pandas,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Коробов Максим Евгеньевич,"С++, Computer Vision, Excel, Git, Go, Jupyter,..."
4,81,10pandas,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Назербаев Руслан Бахтиярович,"Back-end разработка, С, CSS, Django, Front-end..."


In [ ]:
df_cleaned = df_cleaned.merge(unique_cases[['case_title', 'required_roles']], on='case_title', how='left')

In [47]:
df_cleaned.shape

(2051, 7)

In [49]:
filtered_df = df_cleaned[df_cleaned['case_title'].isin(unique_cases['case_title'])]

In [53]:
filtered_df.to_csv('historical_data_with_roles.csv', index=False)